In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from transformers import DataCollatorWithPadding
from transformers.trainer_callback import EarlyStoppingCallback, TrainerCallback, TrainerState, TrainerControl
from datasets import Dataset
from sklearn.model_selection import train_test_split

from tokenizers import (
    decoders,
    models,
    normalizers,
    pre_tokenizers,
    processors,
    trainers,
    Tokenizer,
)

/kaggle/input/chess/games.csv


2024-03-10 17:04:06.043286: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-10 17:04:06.043385: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-10 17:04:06.182177: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
data = pd.read_csv('/kaggle/input/chess/games.csv')
games = data[['winner', 'turns', 'white_rating', 'black_rating', 'moves', 'opening_eco']]
games = games[games['winner']!='draw']

In [4]:
id2label = {0: "white", 1: "black"}
label2id = {"white": 0, "black": 1}

model = AutoModelForSequenceClassification.from_pretrained('bert-base-cased', id2label=id2label, label2id=label2id)
tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')
tokenizer.pre_tokenizer = pre_tokenizers.WhitespaceSplit()

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

In [5]:
def preprocess_moves(examples):
    for i in range(len(examples)):
        temp = examples[i].split(' ')
        examples[i] = " ".join(temp[:20])
    return examples

def preprocess(example):
    tokenized_example = tokenizer(example['moves'])
    tokenized_example['label'] = label2id[example['winner']]
    return tokenized_example

In [6]:
games['moves'] = preprocess_moves(list(games['moves']))

In [7]:
X_train, X_test, y_train, y_test = train_test_split(games['moves'], games['winner'], test_size=0.2, random_state=40)
train = pd.concat([X_train, y_train], axis=1)
train_ds = Dataset.from_pandas(train)
test = pd.concat([X_test, y_test], axis=1)
test_ds = Dataset.from_pandas(test)
tokenized_train = train_ds.map(preprocess)
tokenized_train = tokenized_train.remove_columns(['winner'])
tokenized_valid = test_ds.map(preprocess)
tokenized_valid = tokenized_valid.remove_columns(['winner'])

  0%|          | 0/15286 [00:00<?, ?ex/s]

  0%|          | 0/3822 [00:00<?, ?ex/s]

In [64]:
import evaluate

accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [66]:
trained_model_dir_path = '/kaggle/working'
output_dir = trained_model_dir_path + 'tts'
best_model_dir = trained_model_dir_path + '/best'



training_args = TrainingArguments(
    output_dir=output_dir,
    load_best_model_at_end=True,
    save_total_limit=1,
    evaluation_strategy='epoch',
    save_strategy='epoch',
    warmup_ratio=0.2,
    learning_rate=1e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=8,
    report_to='none',
    weight_decay=0.03,
    lr_scheduler_type='cosine'
)

model = AutoModelForSequenceClassification.from_pretrained('bert-base-cased', id2label=id2label, label2id=label2id)

trainer = Trainer(
    model=model,
    args=training_args,
    tokenizer=tokenizer,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_valid,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=5)],
    compute_metrics=compute_metrics
)

trainer.train()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.684974,0.546049
2,0.695800,0.677539,0.565149
3,0.688000,0.665754,0.566719
4,0.672400,0.656042,0.583203
5,0.660900,0.657224,0.592098
6,0.644900,0.670932,0.590267
7,0.627100,0.682490,0.596808
8,0.613300,0.685620,0.597070


Checkpoint destination directory /kaggle/workingtts/checkpoint-2868 already exists and is non-empty. Saving will proceed but saved results may be invalid.


TrainOutput(global_step=3824, training_loss=0.6537001182843452, metrics={'train_runtime': 887.6866, 'train_samples_per_second': 137.76, 'train_steps_per_second': 4.308, 'total_flos': 4049289419766000.0, 'train_loss': 0.6537001182843452, 'epoch': 8.0})

In [3]:
import scipy


In [9]:
.597070 * 3822

2282.0015399999997

In [12]:
1-scipy.stats.binom.cdf(k=2282, n=3822, p=0.5)

0.0